###Kvasir dataset 14 classes and its iamges


In [ ]:
import os

class_dist = {}
def image_counter(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png']

    for root, dirs, _ in os.walk(folder_path):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            count = 0

            for filename in os.listdir(dir_path):
                file_ext = os.path.splitext(filename)[1].lower()

                if file_ext in image_extensions:
                    count += 1

            class_dist[dir_name] = count

    return class_dist

# Set the path to your dataset directory
dataset_dir = '/content/drive/MyDrive/Capsule Endoscopy-images/'

# Call the image_counter function to count images in each class
class_counts = image_counter(dataset_dir)

# Print the count of images in each class
for class_name, count in class_counts.items():
    print(f"Class '{class_name}': {count} images")


Class 'Ampulla of vater': 10 images
Class 'Blood - hematin': 12 images
Class 'Angiectasia': 866 images
Class 'Blood - fresh': 446 images
Class 'Erosion': 506 images
Class 'Erythema': 159 images
Class 'Foreign body': 776 images
Class 'Ileocecal valve': 4189 images
Class 'Lymphangiectasia': 592 images
Class 'Normal clean mucosa': 34335 images
Class 'Polyp': 55 images
Class 'Pylorus': 1529 images
Class 'Reduced mucosal view': 2906 images
Class 'Ulcer': 854 images


#### Feature Extraction using DenseNet 121 Architecture to all 14 classes

In [ ]:
  import os
  import numpy as np
  from tensorflow.keras.applications import DenseNet121
  from tensorflow.keras.models import Model
  from tensorflow.keras.layers import GlobalAveragePooling2D
  from tensorflow.keras.preprocessing import image
  from tensorflow.keras.applications.densenet import preprocess_input

  # Set the paths
  input_dir ='/content/drive/MyDrive/Capsule Endoscopy-images/'
  output_dir = '/content/drive/MyDrive/FEATURES'

  # Create a DenseNet-121 model (pre-trained on ImageNet)
  base_model = DenseNet121(weights='imagenet', include_top=False)


  feature_layer = GlobalAveragePooling2D()(base_model.output)
  model = Model(inputs=base_model.input, outputs=feature_layer)

  # Loop through each class folder
  class_names = os.listdir(input_dir)
  for class_name in class_names:
      # Set the paths for input and output folders
      input_class_dir = os.path.join(input_dir, class_name)
      output_class_dir = os.path.join(output_dir, class_name)

      # Create the output directory if it doesn't exist
      os.makedirs(output_class_dir, exist_ok=True)

      # Get a list of image files in the input class folder
      image_files = [f for f in os.listdir(input_class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

      # Loop through each image in the class folder
      for image_file in image_files:
          # Load and preprocess the input image
          img_path = os.path.join(input_class_dir, image_file)
          img = image.load_img(img_path, target_size=(224, 224))  # Ensure the target size matches the DenseNet-121 input size
          x = image.img_to_array(img)
          x = preprocess_input(x)
          x = np.expand_dims(x, axis=0)

          # Extract features from the input image
          features = model.predict(x)

          # Save the extracted feature vector as a numpy file
          output_feature_filename = os.path.splitext(image_file)[0] + '_features.npy'
          output_feature_path = os.path.join(output_class_dir, output_feature_filename)
          np.save(output_feature_path, features)

  print("Feature extraction completed.")


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 28ms/step


####Using the minute CNN 2d model to traing the model splitting 0.8:0.2 as train and val

In [ ]:
import os
import numpy as np

feature_dir = '/content/drive/MyDrive/FEATURES'  # Update this to your feature directory
class_folders = os.listdir(feature_dir)

all_features = []
all_labels = []

for class_folder in class_folders:
    class_path = os.path.join(feature_dir, class_folder)
    features = [np.load(os.path.join(class_path, feature_file), allow_pickle=True) for feature_file in os.listdir(class_path) if feature_file.endswith('.npy')]
    labels = [class_folder] * len(features)  # Assign the class folder name as the label

    all_features.extend(features)
    all_labels.extend(labels)

# Convert the lists to NumPy arrays
features_array = np.array(all_features)
labels_array = np.array(all_labels)



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_array, labels_array, test_size=0.2)

# Create a logistic regression model
model = LogisticRegression()

# Train the logistic regression model
model.fit(X_train, y_train)

# Evaluate the logistic regression model on the test set
score = model.score(X_test, y_test)
print("Accuracy:", score)


NameError: ignored